## Pre-clustering with K-means

### tf, df, dictionary

In [1]:
from collections import Counter

def dic_(allDocument): # alldocument=collection: list
    long_str = " ".join(allDocument)
    b=list(set(list(long_str.split())))
    return sorted(b)

def tf_(doc_que): # term frequency
    counts = Counter(list(doc_que.split()))
    return dict(counts)

def df_(term, allDocuments):  # df: no. of occurance of a term in whole collection
    dic=dict.fromkeys(term, 0)
    for i in allDocuments:
        n=0
        for word in term:
            if word in i.split():         
                dic[word]=dic[word]+1
            n+=1  
    return dic  

### tf-idf

In [2]:
import math

def tfidf_(query_s,c,df,allDocument): # query_s: Series (df['col']), allDocument:list

    all_tfidf=[]
    for query in query_s:
        q_tf=tf_(query)
        max_tf=max(q_tf.values())
        tfidf=[]
        for word in c:
            if word not in q_tf: # query item does not shown in doc collection
                    value=0
            else:
                value=(1+math.log10(q_tf.get(word)))/(1+math.log10(max_tf))*math.log10(len(allDocument)/df.get(word))
            tfidf.append(value)
        all_tfidf.append(tfidf)
    return np.array(all_tfidf) #vector

### cosine similarity

In [3]:
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a,b):
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    return cos_sim

### Import Data

In [4]:
import pandas as pd
import numpy as np

df_doc=pd.read_excel('df_doc_pre.xlsx')
df_query=pd.read_excel('df_query_pre.xlsx')
doc_col=df_doc.loc[: , "Text_tok"].tolist() #collection: list

In [5]:
dic_df=pd.read_csv('dictionary.csv')
doc_fre_df=pd.read_csv('document_frequency.csv')
dic=dic_df['0'].tolist()
doc_fre=pd.Series(doc_fre_df.df.values,index=doc_fre_df.term).to_dict()

### tfidf

In [6]:
d_tfidf=tfidf_(df_doc["Text_tok"],dic,doc_fre,doc_col) # doc tf-idf
q_tfidf=tfidf_(df_query["Text_tok"],dic,doc_fre,doc_col) # query tf-idf

### k-means

In [7]:
import numpy as np

# randomly select initial doc leaders
def initialize_centroids(points, k, ran_seed):
    np.random.seed(seed=ran_seed)
    centroids = points.copy()
    np.random.shuffle(centroids)
    return centroids[:k]

# assign each doc to cloest leader cluster
def closest_centroid(points, centroids):
    cluster_label=[]
    for doc in range(len(points)):
        similarity=[]
        for l in range(len(centroids)):
            sim=cosine_similarity(points[doc],centroids[l])
            similarity.append(sim)
        cluster_label.append(np.argmax(similarity))
    return np.asarray(cluster_label)

# adjust doc leaders by take centroid of initial cluster
def move_centroids(points, closest, centroids):
    mean=[]
    for k in range(centroids.shape[0]):
        if len(d_tfidf[closest==k])==0:
            mean.append([0.5])
        else:
            mean.append(d_tfidf[closest==k].mean(axis=0))
        move_centroid= np.zeros([len(mean),len(max(mean,key = lambda x: len(x)))])
        for i,j in enumerate(mean):
            move_centroid[i][0:len(j)] = j
    return move_centroid
    #return np.array([points[closest==k].mean(axis=0) for k in range(centroids.shape[0])])

# recurrently adjust cluster until no movement among clusters
def k_means(points, k,ran_seed):
    leaders=initialize_centroids(points, k, ran_seed)
    move=True
    while move != False:
        leaders_new=move_centroids(points, closest_centroid(points, leaders), leaders)
        for i in range(len(leaders)):
            if cosine_similarity(leaders[i],leaders_new[i])>0.99999:
                move=False
            else:
                move=True
        leaders=leaders_new
    return leaders, closest_centroid(points, leaders) # cluster label

# integrated call k-means for all doc file
def pre_clustering(doc,k,ran_seed):
    cluster=k_means(doc, k, ran_seed)
    leader=cluster[0]
    cluster_label=cluster[1]
    df_d_tfidf = pd.DataFrame(doc)
    df_d_tfidf['label']=cluster_label
    return leader, df_d_tfidf

### Implementation: Cosine Similarity

In [8]:
import math
import operator
import time

def cosSimilarityImp(q_tfidf,leader, df_d_tfidf):
    
    start = time.time()
    sim_all={}
    for q in range(len(q_tfidf)):
        leader_sim={}
        sim={}
        sim_all[q]=sim
        
        # compare similarity between query and doc leaders
        for l in range(len(leader)):
            leader_sim[l]=cosine_similarity(leader[l],q_tfidf[q])
        
        # access the cloest leader's cluster
        closest_cluster_label=max(leader_sim.items(), key=operator.itemgetter(1))[0]
        closest_cluster=df_d_tfidf.loc[df_d_tfidf['label'] == closest_cluster_label].drop(['label'],axis=1)
        
        # calculate similarity amongth docs in the cloest cluster
        for d in closest_cluster.index:
            sim[d]=cosine_similarity(closest_cluster.loc[d],q_tfidf[q])

    end = time.time()
    print('similarity time: ',end - start)

    # ranking doc by similartiy
    # covert format to a table
    df_similarity= pd.DataFrame(data=sim_all)
    df_similarity['d_index']=df_similarity.index
    df1 = df_similarity.reset_index(drop=True)
    df2 = pd.melt(df1, id_vars=["d_index"], var_name="q_index", value_name="similarity")
    df2=df2.sort_values(by=['q_index','similarity'],ascending=[True,False])
    
    # prepare format for meaturement
    df_query['q_index']=df_query.index
    df_doc['d_index']=df_doc.index
    df2=pd.merge(df2,df_query[['Query id','q_index']],on='q_index',how='left')
    df2=pd.merge(df2,df_doc[['Doc id','d_index']],on='d_index',how='left')
    df_similarity_final=df2[['Query id', 'Doc id', 'similarity']]
    df_similarity_final=df_similarity_final.dropna()

    # measuremnt format
    rel_label = pd.read_excel("all2-1-0.qrel.xlsx")
    rel_similarity=pd.merge(df_similarity_final[['Query id','Doc id']],rel_label, on=['Query id','Doc id'], how='left')
    rel_similarity=rel_similarity.fillna(value=0)
    rel_similarity=rel_similarity.drop(['Doc id'],axis=1)
    s = rel_similarity.groupby('Query id')['Rel_level'].apply(lambda x: x.tolist())
    correct_input=s.values

    return correct_input

#### MAP

In [9]:
import numpy as np 
# presicion 
def precision(y_true):
#y_true: this list reordered y_true list
    p=[]
    a=0
    for i in range(len(y_true)):
        if y_true[i]==1:
            a+=1
            precision=a/(i+1)
            p.append(precision)
    #print(p)
    return p

# average precision
def AP(y_true):
    p=precision(y_true)
    if len(p)!=0:
        AP=sum(p)/len(p)
    else:
        AP=0
    #print(AP)
    return AP


def MAP(list_true):# query list 
    ap=[]
    for i in range(len(list_true)):
        ap.append(AP(list_true[i]))
    total=sum(ap)
    #print(ap)
    return total/(len(list_true))

#### nDCG1

In [10]:
import numpy as np 
def ndcg1(correct):
    if sum(correct)==0:
        return 0
    else:
        dcg=0
        for i in range(len(correct)):
            gain=2**correct[i]-1
            discounts=np.log2(i+2)
            dcg=dcg+gain/discounts
    
        #idcg
        order = np.argsort(correct)
        sort_correct = np.take(correct, order[::-1])
        idcg=0
        for i in range(len(sort_correct)):
            gain=2**sort_correct[i]-1
            discounts=np.log2(i+2)
            idcg=idcg+gain/discounts
        #print(dcg)
        #print(idcg)
        return dcg/idcg

def mean_ndcg1(correct_list):
    b=0
    for correct in correct_list:
        a=ndcg1(correct)
        b=b+a
    return b/len(correct_list)

### Cluster Tuning

In [71]:
leader_size=[round(math.sqrt(len(df_doc))),10,20,30,40,50,60,70,80,90,100]
for i in leader_size:
    print('leader size:',i)
    cluster=pre_clustering(d_tfidf,i,0)
    result=cosSimilarityImp(q_tfidf,cluster[0], cluster[1])
    print('MAP:',MAP(result)) 
    print('nDCG:',mean_ndcg1(result))
    print()

leader size: 73
similarity time:  130.38399982452393
MAP: 0.16647725211744085
nDCG: 0.332509782158

leader size: 10
similarity time:  713.3910000324249
MAP: 0.14071103444505317
nDCG: 0.366328603313

leader size: 20
similarity time:  398.2190001010895
MAP: 0.15549867223332753
nDCG: 0.371494921929

leader size: 30
similarity time:  263.15000009536743
MAP: 0.14598624816610853
nDCG: 0.32956926687

leader size: 40
similarity time:  208.57400012016296
MAP: 0.1499140295206088
nDCG: 0.324891392085

leader size: 50
similarity time:  203.88299989700317
MAP: 0.1608903085845128
nDCG: 0.333404291404

leader size: 60
similarity time:  159.4539999961853
MAP: 0.1706197191414928
nDCG: 0.343818173453

leader size: 70
similarity time:  143.66100001335144
MAP: 0.1677409713368592
nDCG: 0.336407876822

leader size: 80
similarity time:  122.69800019264221
MAP: 0.16873337677021938
nDCG: 0.334669620124

leader size: 90
similarity time:  118.66099977493286
MAP: 0.17147095218919614
nDCG: 0.338990584298

leader s

In [82]:
leader_size=[110,120,130]
for i in leader_size:
    print('leader size:',i)
    cluster=pre_clustering(d_tfidf,i,0)
    result=cosSimilarityImp(q_tfidf,cluster[0], cluster[1])
    print('MAP:',MAP(result)) 
    print('nDCG:',mean_ndcg1(result))
    print()

leader size: 110
similarity time:  106.97599983215332
MAP: 0.17466469715793795
nDCG: 0.340284182768

leader size: 120
similarity time:  104.0990002155304
MAP: 0.17644108208323311
nDCG: 0.341795856724

leader size: 130
similarity time:  89.58999991416931
MAP: 0.17500583360650274
nDCG: 0.334936568678



In [330]:
leader_size=[140,150,160,170,180,190,200]
for i in leader_size:
    print('leader size:',i)
    cluster=pre_clustering(d_tfidf,i,0)
    result=cosSimilarityImp(q_tfidf,cluster[0], cluster[1])
    print('MAP:',MAP(result)) 
    print('nDCG:',mean_ndcg1(result))
    print()

leader size: 140
similarity time:  88.0
MAP: 0.17942545037657873
nDCG: 0.339927571527

leader size: 150
similarity time:  85.20500016212463
MAP: 0.1877172936437659
nDCG: 0.349451203786

leader size: 160
similarity time:  153.2464997768402
MAP: 0.1861306809915443
nDCG: 0.335513375168

leader size: 170
similarity time:  80.12099981307983
MAP: 0.17900668129252661
nDCG: 0.323470708261

leader size: 180
similarity time:  75.92799997329712
MAP: 0.18895744958751873
nDCG: 0.340065755626

leader size: 190
similarity time:  99.0770001411438
MAP: 0.1918335052238354
nDCG: 0.350583086344

leader size: 200
similarity time:  97.99000000953674
MAP: 0.19413169712082684
nDCG: 0.355728402021



In [331]:
leader_size=[210,220,230,240,250,260,270,280,290,300]
for i in leader_size:
    print('leader size:',i)
    cluster=pre_clustering(d_tfidf,i,0)
    result=cosSimilarityImp(q_tfidf,cluster[0], cluster[1])
    print('MAP:',MAP(result)) 
    print('nDCG:',mean_ndcg1(result))
    print()

leader size: 210
similarity time:  84.76899981498718
MAP: 0.18903357735855497
nDCG: 0.344107942425

leader size: 220
similarity time:  84.1470000743866
MAP: 0.1901799031129375
nDCG: 0.346668773164

leader size: 230
similarity time:  85.21300029754639
MAP: 0.19647131914645843
nDCG: 0.356512201186

leader size: 240
similarity time:  83.21600008010864
MAP: 0.19953938571476806
nDCG: 0.356800576444

leader size: 250
similarity time:  86.43899989128113
MAP: 0.19399494724555605
nDCG: 0.351518256047

leader size: 260
similarity time:  84.73699998855591
MAP: 0.19198471706786763
nDCG: 0.346114078491

leader size: 270
similarity time:  84.90200018882751
MAP: 0.19919271505162595
nDCG: 0.356598269705

leader size: 280
similarity time:  83.6159999370575
MAP: 0.19598083242870845
nDCG: 0.351317699869

leader size: 290
similarity time:  69.45799994468689
MAP: 0.19772726226067341
nDCG: 0.350784532249

leader size: 300
similarity time:  71.48499989509583
MAP: 0.20065772769139037
nDCG: 0.357306195665



In [11]:
leader_size=[310,320,330,340,350,360,370,380,390,400]
for i in leader_size:
    print('leader size:',i)
    cluster=pre_clustering(d_tfidf,i,0)
    result=cosSimilarityImp(q_tfidf,cluster[0], cluster[1])
    print('MAP:',MAP(result)) 
    print('nDCG:',mean_ndcg1(result))
    print()

leader size: 310
similarity time:  66.46840023994446
MAP: 0.19995321502849656
nDCG: 0.359862948451

leader size: 320
similarity time:  64.26240038871765
MAP: 0.2001392196866568
nDCG: 0.356806226464

leader size: 330
similarity time:  67.57520008087158
MAP: 0.2018556793308341
nDCG: 0.358794050847

leader size: 340
similarity time:  68.06320023536682
MAP: 0.20453057735783822
nDCG: 0.361440821342

leader size: 350
similarity time:  67.9992003440857
MAP: 0.2086312621173508
nDCG: 0.364418089138

leader size: 360
similarity time:  70.13760042190552
MAP: 0.20571362472854066
nDCG: 0.36191018093

leader size: 370
similarity time:  66.26280045509338
MAP: 0.20899772728906782
nDCG: 0.36500915294

leader size: 380
similarity time:  67.4558002948761
MAP: 0.2123166904399022
nDCG: 0.369244856697

leader size: 390
similarity time:  71.2746000289917
MAP: 0.2129859843112712
nDCG: 0.368860704528

leader size: 400
similarity time:  70.76460027694702
MAP: 0.2132481427708949
nDCG: 0.367020541802



In [12]:
leader_size=[1000]
for i in leader_size:
    print('leader size:',i)
    cluster=pre_clustering(d_tfidf,i,0)
    result=cosSimilarityImp(q_tfidf,cluster[0], cluster[1])
    print('MAP:',MAP(result)) 
    print('nDCG:',mean_ndcg1(result))
    print()

leader size: 1000
similarity time:  109.51120042800903
MAP: 0.23940693775627106
nDCG: 0.385937226586



In [13]:
leader_size=[2000,3000,4000,5000]
for i in leader_size:
    print('leader size:',i)
    cluster=pre_clustering(d_tfidf,i,0)
    result=cosSimilarityImp(q_tfidf,cluster[0], cluster[1])
    print('MAP:',MAP(result)) 
    print('nDCG:',mean_ndcg1(result))
    print()

leader size: 2000
similarity time:  199.09560084342957
MAP: 0.2709207573606207
nDCG: 0.423240905447

leader size: 3000
similarity time:  287.1690020561218
MAP: 0.2641303910279825
nDCG: 0.418096768901

leader size: 4000
similarity time:  372.91960167884827
MAP: 0.2657620101897219
nDCG: 0.411918180112

leader size: 5000
similarity time:  461.2810027599335
MAP: 0.26014166556335266
nDCG: 0.405423213506



In [11]:
leader_size=[500,600,700,800,900]
for i in leader_size:
    print('leader size:',i)
    cluster=pre_clustering(d_tfidf,i,0)
    result=cosSimilarityImp(q_tfidf,cluster[0], cluster[1])
    print('MAP:',MAP(result)) 
    print('nDCG:',mean_ndcg1(result))
    print()

leader size: 500
similarity time:  69.35350036621094
MAP: 0.21483842387504723
nDCG: 0.365105603717

leader size: 600
similarity time:  73.99410009384155
MAP: 0.22015704148524173
nDCG: 0.372103523303

leader size: 700
similarity time:  83.1092004776001
MAP: 0.21996053393525491
nDCG: 0.371516094735

leader size: 800
similarity time:  88.52700066566467
MAP: 0.2265230458335907
nDCG: 0.379137753152

leader size: 900
similarity time:  97.65310049057007
MAP: 0.23410509348729133
nDCG: 0.384326388194

